In [1]:
!nvidia-smi

Sun Oct 22 10:43:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
HOME = os.getcwd()
print(HOME)

%cd {HOME}
!git clone -q https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/content
/content
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/content
/content/weights
/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [4]:
%cd {HOME}
!mkdir {HOME}/data
%cd {HOME}/data

!wget -q https://media.roboflow.com/notebooks/examples/dog.jpeg
!wget -q https://media.roboflow.com/notebooks/examples/dog-2.jpeg
!wget -q https://media.roboflow.com/notebooks/examples/dog-3.jpeg
!wget -q https://media.roboflow.com/notebooks/examples/dog-4.jpeg

/content
/content/data


In [5]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

/content/GroundingDINO


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


In [6]:
import numpy as np
from PIL import Image
import groundingdino.datasets.transforms as T

def inference(img, prompt, box_threshold=0.35, text_threshold=0.25):
    transform = T.Compose([
        T.RandomResize([800], max_size=1333),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    image_transformed, _ = transform(Image.fromarray(img), None)

    boxes, logits, phrases = predict(
        model=model,
        image=image_transformed,
        caption=prompt,
        box_threshold=box_threshold,
        text_threshold=text_threshold
    )

    annotated_frame = annotate(image_source=img, boxes=boxes, logits=logits, phrases=phrases)

    return annotated_frame

In [7]:
!pip install flask-ngrok
!pip install flask

In [8]:
!mkdir /root/.ngrok2 && echo "authtoken: 2Qw7vzk6LAuODYrr9JR7JgTiao2_6WbTPYgiELuEcbrWPVRYs" > /root/.ngrok2/ngrok.yml

In [10]:
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
import os
import cv2
import base64
import numpy as np  # You missed importing numpy
from google.colab import files
from concurrent.futures import ThreadPoolExecutor

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def hello():
    return "Success"

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' in request.files and 'prompt' in request.form:
        print("전송완료")
        photo = request.files['file']
        prompt = request.form['prompt']

        img = cv2.imdecode(np.fromstring(photo.read(), np.uint8), cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        result_img = inference(img, prompt)  # Assuming that inference() is some function you have defined elsewhere

        _, buffer = cv2.imencode('.jpg', result_img)
        jpg_as_text = base64.b64encode(buffer).decode()

        return jsonify({'result_image': jpg_as_text})

    else:
        return jsonify({'error': '사진 또는 프롬프트가 업로드되지 않았습니다.'})

def process_frame(args):
    frame, prompt = args
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result_img = inference(frame, prompt)
    return result_img

@app.route('/uploadvideo', methods=['POST'])
def upload_videofile():
    if 'file' in request.files and 'prompt' in request.form:
        video = request.files['file']
        prompt = request.form['prompt']

        # Save the uploaded video file
        filepath = os.path.join("/content", video.filename)
        video.save(filepath)

        cap = cv2.VideoCapture(filepath)
        frames = []
        while cap.isOpened():
            ret, img = cap.read()
            if not ret:
                break
            frames.append(img)

        output_path = '/content/output.mp4'
        fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
        out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                              (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

        with ThreadPoolExecutor() as executor:
            # Process frames in parallel
            results = list(executor.map(process_frame, [(frame, prompt) for frame in frames]))

        for result in results:
            out.write(result)

        out.release()
        cap.release()

        return jsonify({'message': 'Video processed successfully!', 'video_path': '/getvideo'})

    else:
        return jsonify({'error': '비디오 또는 프롬프트가 업로드되지 않았습니다.'})


@app.route('/getvideo', methods=['GET'])
def get_video():
    return send_file('/content/output.mp4', as_attachment=True, download_name='output.mp4')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://790f-34-87-103-7.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:03:53] "GET / HTTP/1.1" 200 -
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:905: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:06:19] "POST /uploadvideo HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22

전송완료


<ipython-input-10-87687fdaecba>:24: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = cv2.imdecode(np.fromstring(photo.read(), np.uint8), cv2.IMREAD_UNCHANGED)
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:12:28] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:15:50] "POST /uploadvideo HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:15:51] "GET /getvideo HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:15:52] "GET /getvideo HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:15:53] "GET /getvideo HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:15:54] "GET /getvideo HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:16:12] "GET /getvideo HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:16:14] "GET /getvideo HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2023 11:16:17] "GET /getvideo HTTP/1.1" 206 -


In [ ]:
!pip install flask-ngrok
!pip install flask